In [1]:
import mlflow

In [2]:
!mlflow --version

mlflow, version 1.26.0


In [3]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet -O ../data/green_tripdata_2021-01.parquet
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet -O ../data/green_tripdata_2021-02.parquet
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet -O ../data/green_tripdata_2021-03.parquet

--2022-05-30 20:57:07--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.232.128
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.232.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘../data/green_tripdata_2021-01.parquet’

../data/green_tripd 100%[===================>]   1.27M   640KB/s    in 2.0s    

2022-05-30 20:57:11 (640 KB/s) - ‘../data/green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2022-05-30 20:57:11--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.232.128
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.232.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145679 (1.1M) [binary/octet-stream]
Saving to: ‘../data/green_tripdata_2021-02.parquet’

../data/green_tripd 100%[===========

In [4]:
!pwd

/home/mikeyy/my-mlops-zoomcamp/week-2


# Question 2: Number of files in output folder
**4**

In [5]:
ls helper_codes/

hpo.py  preprocess_data.py  register_model.py  train.py


In [6]:
!python helper_codes/preprocess_data.py --raw_data_path ../data/ --dest_path ./output

In [7]:
ls output

dv.pkl  test.pkl  train.pkl  valid.pkl


# Question 3: Number of parameters logged with autolog
**17**


In [8]:
!python helper_codes/train.py --data_path ./output

./output
2022/05/30 20:57:26 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/30 20:57:26 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e246ab4e748c4f0a9caf1b451997c18e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/05/30 20:57:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/mikeyy/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


# Question 4: Parameter for launching the tracking server
In addition to backend-store-uri, what else do you need to pass to properly configure the server?
<br>
**default-artifact-root**

In [9]:
ls

helper_codes/     LearningNotebook.ipynb  mlruns/  requirements.txt
Homework-2.ipynb  mlflow.db               output/  running-mlflow-examples/


# Question 5: RMSE with hyperopt
**6.628**

In [73]:
!python helper_codes/hpo.py --data_path ./output

100%|█████████| 50/50 [06:42<00:00,  8.05s/trial, best loss: 6.6284257482044735]


# Question 6: RMSE on test

In [74]:
from mlflow.tracking import MlflowClient

In [75]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [76]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment-homework-week2', tags={}>,
 <Experiment: artifact_location='./mlruns/5', experiment_id='5', lifecycle_stage='active', name='random-forest-hyperopt-1', tags={}>,
 <Experiment: artifact_location='./mlruns/6', experiment_id='6', lifecycle_stage='active', name='random-forest-best-models-1', tags={}>]

In [77]:
HPO_EXPERIMENT_NAME = "random-forest-hyperopt-1"

In [78]:
experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
experiment

<Experiment: artifact_location='./mlruns/5', experiment_id='5', lifecycle_stage='active', name='random-forest-hyperopt-1', tags={}>

In [79]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [80]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 5546266cce0549d486580e366f49b969, rmse: 6.6729
run id: eed83e153a954889ae216010c826fbb2, rmse: 6.6729
run id: 98df72043b9848fb9d17fbcbfb4a48e2, rmse: 7.4478


In [82]:
runs[0].data.params

{'max_depth': '19.0',
 'min_samples_leaf': '3.0',
 'min_samples_split': '5.0',
 'n_estimators': '28.0'}


    # select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    # best_run = client.search_runs( ...  )[0]

    # register the best model
    # mlflow.register_model( ... )

In [28]:
runs[0].info.run_id

'f04f66aa3dd048fcafb3aace1f12fb08'

In [ ]:
run_id = runs[0].info.run_id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-rf-regressor")

In [84]:
!python helper_codes/register_model.py --data_path ./output

2022/05/30 23:14:31 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.




run is: <Run: data=<RunData: metrics={'rmse': 6.6729449097319}, params={'max_depth': '19.0',
 'min_samples_leaf': '3.0',
 'min_samples_split': '5.0',
 'n_estimators': '28.0'}, tags={'mlflow.source.git.commit': '393e344149cfe6e29bc3d8d520b424e18acae2c4',
 'mlflow.source.name': 'helper_codes/hpo.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'mikeyy'}>, info=<RunInfo: artifact_uri='./mlruns/5/5546266cce0549d486580e366f49b969/artifacts', end_time=1653931533902, experiment_id='5', lifecycle_stage='active', run_id='5546266cce0549d486580e366f49b969', run_uuid='5546266cce0549d486580e366f49b969', start_time=1653931143164, status='FINISHED', user_id='mikeyy'>>




2022/05/30 23:14:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encounter